## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_3 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o3/o3_bincore6.csv", index_col=0)
print(bin6_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_3.reset_index(inplace=True, drop=True)

print('shape -', bin6_3.shape)
print('reset_index 완료')
print('input data shape')
bin6_3.head()

(33250757, 2)
shape - (33250757, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_3['label'].value_counts())

256
0    33215560
1       35197
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_3[bin6_3['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 48 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_3), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_3', len(ls_idx_bin))

# loc 로 수정필요
bin6_3_Ngram = bin6_3.loc[ls_idx_bin,:].copy()

1689456
나머지 0
최종 길이 1689456
bin6_3 1689456


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_3_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_3.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_3[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

35197.0
0 4207975
1000 30119337
2000 2553101
3000 11440055
4000 18368044
5000 13651356
6000 20069418
7000 7581010
8000 2417800
9000 13539467
10000 30153662
11000 6799709
12000 32146518
13000 22303725
14000 28044061
15000 30712170
16000 11277561
17000 5803796
18000 7415743
19000 7451503
20000 25539277
21000 2421688
22000 31284350
23000 357298
24000 17438250
25000 20254575
26000 16603594
27000 24949516
28000 25250190
29000 12002801
30000 12498259
31000 1954220
32000 5528192
33000 31894292
34000 29744326
35000 21572399
완료
35197


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_3['label'].value_counts()[1])

35197
35197


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_3_Ngram])
final.shape

(3378912, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_3_onehot_Ngram = pd.concat([final['label'], bc6_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_3_onehot_Ngram.shape)

원핫인코딩완료
(3378912, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_3 = bc6_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_3 = bc6_3_onehot_Ngram['label'].to_numpy()
print(x_bc6_3.shape, x_bc6_3.shape)

x_bc6_3 = x_bc6_3.reshape(-1, right_idx, x_bc6_3.shape[1])
y_bc6_3 = y_bc6_3.reshape(-1, right_idx, 1)

print(x_bc6_3.shape, y_bc6_3.shape)

(3378912, 256) (3378912, 256)
(70394, 48, 256) (70394, 48, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_3.shape[0])

x_bc6_3 = x_bc6_3[p]
y_bc6_3 = y_bc6_3[p]

print(x_bc6_3.shape, y_bc6_3.shape)

(70394, 48, 256) (70394, 48, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(72, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_3, y_bc6_3):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_3[train],
               y_bc6_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 210s 3ms/sample - loss: 0.0090 - accuracy: 0.9975
Epoch 2/10
63354/63354 [==============================] - 228s 4ms/sample - loss: 0.0011 - accuracy: 0.9997
Epoch 3/10
63354/63354 [==============================] - 228s 4ms/sample - loss: 7.0021e-04 - accuracy: 0.9998
Epoch 4

accuracy_score 0.999908262310606
recall_score 0.9948550046772685
precision_score 0.9978888106966924
f1_score 0.9963695983136199
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 222s 4ms/sample - loss: 1.9197e-04 - accuracy: 0.9999
Epoch 2/10
63354/63354 [==============================] - 218s 3ms/sample - loss: 1.5567e-04 

accuracy_score 0.9999230587121212
recall_score 0.9990643274853801
precision_score 0.9948753785231773
f1_score 0.9969654528478057
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 277s 4ms/sample - loss: 7.3095e-05 - accuracy: 1.0000
Epoch 2/10
63354/63354 [==============================] - 187s 3ms/sample - loss: 7.1206e-05

accuracy_score 0.9999792850378788
recall_score 0.998837479655894
precision_score 0.9995346672871103
f1_score 0.9991859518548668
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 224s 4ms/sample - loss: 4.0733e-05 - accuracy: 1.0000
Epoch 2/10
63354/63354 [==============================] - 137s 2ms/sample - loss: 3.7377e-05 

accuracy_score 0.9999733664772728
recall_score 0.9997644287396937
precision_score 0.9981185324553151
f1_score 0.9989408026362245
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 178s 3ms/sample - loss: 2.5808e-05 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 145s 2ms/sample - loss: 2.6497e-05

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 160s 3ms/sample - loss: 1.0795e-05 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 149s 2ms/sample - loss: 1.0093e-05 - accuracy: 1.0000
Epoch 3/10
63355/63355 [================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 161s 3ms/sample - loss: 5.0733e-06 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 151s 2ms/sample - loss: 5.0728e-06 - accuracy: 1.0000
Epoch 3/10
63355/63355 [================

accuracy_score 0.9999970402992849
recall_score 1.0
precision_score 0.9997638724911452
f1_score 0.9998819223048765
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 171s 3ms/sample - loss: 1.0087e-05 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 133s 2ms/sample - loss: 1.0087e-05 - accuracy: 1.

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 144)           189504    
_________________________________________________________________
time_distributed (TimeDistri (None, 48, 1)             145       
Total params: 189,649
Trainable params: 189,649
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 166s 3ms/sample - loss: 1.0087e-05 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 153s 2ms/sample - loss: 1.0087e-05 - accuracy: 1.0000
Epoch 3/10
63355/63355 [================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.999908262310606, 0.9999230587121212, 0.9999792850378788, 0.9999733664772728, 1.0, 1.0, 1.0, 0.9999970402992849, 1.0, 1.0]

K-fold cross validation Recall: [0.9948550046772685, 0.9990643274853801, 0.998837479655894, 0.9997644287396937, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9978888106966924, 0.9948753785231773, 0.9995346672871103, 0.9981185324553151, 1.0, 1.0, 1.0, 0.9997638724911452, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9963695983136199, 0.9969654528478057, 0.9991859518548668, 0.9989408026362245, 1.0, 1.0, 1.0, 0.9998819223048765, 1.0, 1.0]


## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999781012837163
10-Fold Cross_validation. Recall : 0.9992521240558236
10-Fold Cross_validation. Precision : 0.999018126145344
10-Fold Cross_validation. F1-Score : 0.9991343727957392


In [13]:
model1.save('gcc6_bin_core_s48_h72_o3.h5')
print('save 완료')

save 완료
